UNets
===
MAIC - Spring, Week 4<br>
```
  _____________
 /0   /     \  \
/  \ M A I C/  /\
\ / *      /  / /
 \___\____/  @ /
          \_/_/
```
(Run on Rosie)

In [ ]:
# do this if you can't import helpers
# %cd <DIRECTORY WITH THE WORKSHOP FILES>

In [ ]:
import helpers
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf

## Getting Data

In [ ]:
# Load data

TRAIN_TEST_SPLIT = 0.8

category_names, num_categories = helpers.get_categegory_names()
images, image_categories = helpers.get_data(1)

one_hot_encode = lambda image_categories: tf.keras.utils.to_categorical(image_categories, num_categories)

train_test_split_idx = int(TRAIN_TEST_SPLIT*images.shape[0])
train_imgs, test_imgs = images[:train_test_split_idx], images[train_test_split_idx:]
train_labels, test_labels = one_hot_encode(image_categories[:train_test_split_idx]), one_hot_encode(image_categories[train_test_split_idx:])

## Make & Train the Model

In [ ]:
x = tf.keras.Input(shape=(32,32,3)) ; y=x
y = tf.keras.layers.RandomTranslation((-.1, .1), (-.1, .1))(y)
y = tf.keras.layers.GaussianNoise(0.05)(y)
y = tf.keras.layers.Conv2D(6, 32, strides=1, padding='same', activation='sigmoid')(y)
# skip1 = y
y = tf.keras.layers.Dropout(0.1)(y)
y = tf.keras.layers.Conv2D(4, 16, strides=2, padding='same', activation='sigmoid')(y)
# skip2 = y
y = tf.keras.layers.Dropout(0.1)(y)
y = tf.keras.layers.Conv2D(4, 8, strides=2, padding='same', activation='sigmoid')(y)
# skip3 = y
y = tf.keras.layers.MaxPooling2D(2)(y)
# skip4 = y
y = tf.keras.layers.Flatten()(y)
y = tf.keras.layers.Dense(32, activation='gelu')(y)
y = tf.keras.layers.Dense(16, activation='gelu')(y)
y_label = tf.keras.layers.Dense(10, activation='softmax')(y)
y = tf.keras.layers.Dense(32, activation='gelu')(y)
y = tf.keras.layers.Dense(64, activation='sigmoid')(y)
y = tf.keras.layers.Reshape((4,4,4))(y)
# y = tf.keras.layers.Add()([y, skip4])
y = tf.keras.layers.UpSampling2D(2)(y)
# y = tf.keras.layers.Add()([y, skip3])
y = tf.keras.layers.Conv2DTranspose(4, 8, strides=2, padding='same', activation='sigmoid')(y)
# y = tf.keras.layers.Add()([y, skip2])
y = tf.keras.layers.Dropout(0.1)(y)
y = tf.keras.layers.Conv2DTranspose(6, 16, strides=2, padding='same', activation='sigmoid')(y)
# y = tf.keras.layers.Add()([y, skip1])
y = tf.keras.layers.Dropout(0.1)(y)
y = tf.keras.layers.Conv2DTranspose(3, 32, strides=1, padding='same', activation='sigmoid')(y)

model = tf.keras.Model(inputs=x, outputs=[y, y_label])
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=[tf.keras.losses.mse, tf.keras.losses.categorical_crossentropy])
hist = model.fit(x=train_imgs, y=[train_imgs, train_labels], epochs=40, batch_size=32, validation_data=[test_imgs, [test_imgs, test_labels]])

plt.plot(hist.epoch, hist.history['loss'], hist.history['val_loss'])

## Model Results

In [ ]:
# Separate classifier and reconstructor models

x = tf.keras.Input(shape=(32,32,3))

y_reconstruction = model(x)[0]
y_classification = model(x)[1]

autoencoder = tf.keras.Model(x,y_reconstruction)
classifier  = tf.keras.Model(x,y_classification)

In [ ]:
# Classifier Accuracy
matches = np.argmax(classifier.predict(test_imgs), axis=1) == np.argmax(test_labels, axis=1)
np.sum(matches)/matches.shape[0]

In [ ]:
# Reconstructor qualitative results
input_img = train_imgs[5]
reconstructed_img = autoencoder.predict(input_img[np.newaxis, :])[0]
plt.imshow(input_img)
plt.show()
plt.imshow(reconstructed_img)

In [ ]:
# Checking out the classifier

def predict_class(model, x, y_true):
    img = x
    plt.imshow(img, interpolation='bicubic')

    predicted_category_1he = model.predict(img[np.newaxis, :])[0]
    predicted_category = np.argmax(predicted_category_1he)
    actual_category = np.argmax(y_true)

    print(f'Predicted: {category_names[predicted_category]}\nActual: {category_names[actual_category]}')

predict_idx = 511
predict_class(classifier, test_imgs[predict_idx], test_labels[predict_idx])

### Learned Convolution Kernels

In [ ]:
# show conv layers and their indices
for i, l in enumerate(model.layers):
    if 'conv' not in str(l): continue
    print(f'{i}: {l}')

In [ ]:
kernels = model.layers[21].get_weights()[0]
kernels = np.swapaxes(kernels, 0, 3)
kernels = np.swapaxes(kernels, 2, 3)
kernels.shape

In [ ]:
kernel_to_see = kernels[3,:,:,0:3]
kernel_to_see = np.abs(kernel_to_see)
plt.imshow(kernel_to_see/np.max(kernel_to_see))

# Resources
- Why skip connections? (https://theaisummer.com/skip-connections/)
- More Unet info (https://en.wikipedia.org/wiki/U-Net)
- About stable diffusion (touches on how Unets are used) https://huggingface.co/blog/stable_diffusion
- Feel free to ask questions in "Creations and Findings" and "General" in the MAIC Teams group.

# Challenges
- Add more down/up convolutions with skip connections. (1 challenge point) <span style="color:green">&#9646;&#9646;</span><br>
- Can you get a better test accuracy than we did? (2 challenge points) <span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><br>
- Get reconstructions that are (arguably) recognizable objects. (2 challenge points) <span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><br>
- Train on a different dataset (not necessarily images) (3 challenge points) <span style="color:green">&#9646;&#9646;</span><span style="color:#eedd00">&#9646;&#9646;</span><span style="color:red">&#9646;&#9646;</span><br>